### Word representation
The easiest way to embed a word is one-hot.

$$
\mathbf x =
\begin{bmatrix}
0 \\
0 \\
\vdots \\
1 \\
\vdots \\
0 
\end{bmatrix}_{||V|| \times 1}
$$

#### Analogy of two vectors

$$
\begin{align*}
e_{\text{man}} - e_{\text{woman}} &\approx e_{\text{king}} - e \\
e &\approx e_{\text{king}} - e_{\text{man}} + e_{\text{woman}} \\
\end{align*}
$$

The similarity of the vectors are calculated with cosine similarity.

$$
-1 \leq\mathrm{cossim}(e, e_{\text{king}} - e_{\text{man}} + e_{\text{woman}}) \leq 1
$$

### Embedding matrix

In [169]:
import numpy as np
import matplotlib.pyplot as plt

# embedding matrix
E = np.random.randn(10, 100)

# one-hot vector 
x = np.zeros((100,1))
x[42] += 1

# embedded vector
e = E @ x

### Word2Vec

#### Skip-gram

In [170]:
text = """John quickly realized that the fox was jumping over a brown fence. Meanwhile, the lazy dog slept under the warm sun, dreaming of chasing squirrels in the park. A wizard in a distant land cast spells to levitate objects and summon mystical creatures. The gym was full of athletes lifting weights, running on treadmills, and practicing yoga poses. Buzzing bees were collecting nectar from vibrant flowers, while a group of birds sang harmoniously from the treetops. In the city, cars zoomed by as people hurried to work, their minds filled with tasks and deadlines. The library was a sanctuary of knowledge, where students pored over books and researchers delved into ancient manuscripts. A chef in a bustling kitchen prepared exquisite dishes, skillfully chopping vegetables and grilling meats. At the beach, waves crashed against the shore as children built sandcastles and surfers rode the swells. In the forest, a lumberjack wielded his axe, cutting down trees for timber. The night sky was a tapestry of stars, constellations, and planets, inspiring wonder and awe in all who gazed upon it."""
text = text.lower()

chars = sorted(list(set(text)))
c2i = lambda c: chars.index(c)
i2c = lambda i: chars[i]

X = []
context_size = 2
for i in range(context_size, len(text)-context_size):
    for offset in range(-context_size, context_size+1):
        if offset == 0:
            continue
        X.append([c2i(text[i]), c2i(text[i+offset])])

print(f"{len(text) = }")
print(f"{len(X)    = }")

len(text) = 1093
len(X)    = 4356


In [171]:
import torch
import torch.nn as nn

class SkipGram(nn.Module):
    def __init__(self, vocab_size=len(chars), embedding_dim=5):
        super().__init__()
        self.vocab_size = vocab_size
        self.embedding_dim = embedding_dim

        self.embedding = nn.Embedding(num_embeddings=self.vocab_size,
                                      embedding_dim=self.embedding_dim)
        self.linear = nn.Linear(in_features=self.embedding_dim,
                                out_features=self.vocab_size)
        
        nn.init.xavier_uniform_(self.embedding.weight)
        nn.init.xavier_uniform_(self.linear.weight)
    
    def forward(self, x):
        x = self.embedding(x)
        x = self.linear(x)
        return x

KeyboardInterrupt: 

In [ ]:
import torch.optim as optim
from torch.utils.data import DataLoader

skipgram = SkipGram()
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(skipgram.parameters(), lr=0.1)

skipgram.train()
for e in range(1):
    running_loss = 0
    count = 0
    for x, y in DataLoader(X, batch_size=32, shuffle=True):
        count += 1
        optimizer.zero_grad()

        y_hat = skipgram(x)
        loss = criterion(y_hat, y) # <- computing every skip-grams is expensive. Cross entropy loss of huge vocab size (expensive!) * context size

        loss.backward()
        optimizer.step()

        running_loss += loss.item()
    print(f"[epoch {e+1:0>3}] {running_loss/count:7.5f}", end='\r')

#### Skip-gram (with negative sampling)

In [ ]:
import random

text = """John quickly realized that the fox was jumping over a brown fence. Meanwhile, the lazy dog slept under the warm sun, dreaming of chasing squirrels in the park. A wizard in a distant land cast spells to levitate objects and summon mystical creatures. The gym was full of athletes lifting weights, running on treadmills, and practicing yoga poses. Buzzing bees were collecting nectar from vibrant flowers, while a group of birds sang harmoniously from the treetops. In the city, cars zoomed by as people hurried to work, their minds filled with tasks and deadlines. The library was a sanctuary of knowledge, where students pored over books and researchers delved into ancient manuscripts. A chef in a bustling kitchen prepared exquisite dishes, skillfully chopping vegetables and grilling meats. At the beach, waves crashed against the shore as children built sandcastles and surfers rode the swells. In the forest, a lumberjack wielded his axe, cutting down trees for timber. The night sky was a tapestry of stars, constellations, and planets, inspiring wonder and awe in all who gazed upon it."""
text = text.lower()

chars = sorted(list(set(text)))
c2i = lambda c: chars.index(c)
i2c = lambda i: chars[i]

X = []
context_size = 2
for i in range(context_size, len(text)-context_size):
    target = c2i(text[i])

    positive_samples = []
    for offset in range(-context_size, context_size+1):
        if offset == 0:
            continue
        i = c2i(text[i+offset])
        positive_samples.append(i)

    negative_sample_candidates = list(set(range(len(chars))) - set(positive_samples))
    negative_samples = random.sample(negative_sample_candidates, context_size*2*5) # 1:5 ratio of negative sampling

    for s in positive_samples:
        X.append([target, s, 1])

    for s in negative_samples:
        X.append([target, s, 0])

print(f"{len(text) = }")
print(f"{len(X)    = }")

<img src="https://wikidocs.net/images/page/69141/그림7.PNG" height=300/>

### GloVe (Global vectors for word representation)

Co-occurance of words in a sentence.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

# Define a tiny corpus
corpus = ["I like machine learning",
          "I like coding in Python",
          "I enjoy learning new things"]

vocab = set()
for sentence in corpus:
    vocab.update(sentence.lower().split())
vocab = sorted(list(vocab))

vocab_size = len(vocab)
co_occurrence_matrix = np.zeros((vocab_size, vocab_size))

for sentence in corpus:
    words = sentence.lower().split()
    for i, word1 in enumerate(words):
        for j, word2 in enumerate(words):
            if word1 != word2:
                co_occurrence_matrix[vocab.index(word1), vocab.index(word2)] += 1

fig, ax = plt.subplots(1,4,figsize=(20,4))
ax[0].set_title('co-occurrence matrix')
ax[0].imshow(co_occurrence_matrix, cmap='binary_r')
ax[0].xaxis.set_ticks_position('top')
ax[0].set_xticks(range(len(vocab)))
ax[0].set_xticklabels(vocab, rotation=90)
ax[0].set_yticks(range(len(vocab)))
ax[0].set_yticklabels(vocab)

U, S, V_T = np.linalg.svd(co_occurrence_matrix)
ax[1].set_title('$U$ (word vector)')
ax[1].imshow(U, cmap='binary_r')
ax[1].xaxis.set_ticks_position('top')
ax[1].set_xticks(range(len(vocab)))
ax[1].set_xticklabels(vocab, rotation=90)
ax[1].set_yticks([])

ax[2].set_title('$S$')
ax[2].imshow(np.diag(S), cmap='binary_r')
ax[2].set_xticks([])
ax[2].set_yticks(range(len(vocab)))
ax[2].set_yticklabels(vocab)

ax[3].set_title('$V^\intercal$')
ax[3].imshow(V_T, cmap='binary_r')
ax[3].set_xticks([])
ax[3].set_yticks(range(len(vocab)))
ax[3].set_yticklabels(vocab)

plt.show()


### Debiasing word embeddings

One way to debias some word embeddings (doctor:man=nurse:woman) is to find a non-biased dimension where is perpendicular to gender axis and project them.

<img src="src/debiasing.png" height=400 />

In [ ]:
from pathlib import Path
import pandas as pd

with open('src/glove.6B.50d.txt', 'r') as f:
    rows = f.readlines()

rows = [row.split() for row in rows]
df = pd.DataFrame(rows)
df = df.apply(lambda col: pd.to_numeric(col, errors='ignore'), axis=0)
df.sample(10)

,0,1,2,3,4,5,6,7,8,9,...,41,42,43,44,45,46,47,48,49,50
229985,e.t.c.,0.30635,-1.047300,-0.302170,-0.192570,-1.03850,-0.71002,0.278730,0.338580,-0.147850,...,-0.384140,-1.238200,0.375450,-1.321600,0.158480,0.047650,-0.023359,-0.905180,1.34670,1.27920
289528,2137,-0.27732,-0.156750,-0.028893,-0.114710,-0.57858,-0.68888,0.505910,-0.034682,0.069475,...,0.275080,-0.509240,-0.275620,0.426450,-0.075152,0.505060,0.272850,0.098014,0.27318,0.18083
235861,discoursing,-0.44085,-0.093508,-0.742410,-0.397990,-0.14382,-0.23926,0.092085,-0.087855,-0.098514,...,0.123970,-0.368530,0.024875,0.785340,-0.682100,-0.345760,1.082400,0.386690,0.29955,-1.22470
284668,actualizing,0.10355,0.079821,-0.316390,-0.973870,-0.45961,-0.30537,1.085700,-0.110770,0.599110,...,0.273570,-0.456990,-0.047865,0.421220,0.142690,-0.424830,0.441400,0.327500,0.10860,-0.11897
237748,cubensis,0.29420,-0.569710,-0.479500,0.010294,0.50231,0.19395,0.990630,-0.212240,-0.362770,...,-0.912100,-0.306480,-0.371500,-0.779410,0.202650,0.079314,-0.618540,0.300880,0.30027,1.60140
303066,kezi,-0.12759,-0.227090,0.234360,0.499230,-0.73960,0.31408,-0.008921,0.595970,0.127430,...,0.210970,-0.051466,-0.225720,-0.334390,-0.249560,0.265710,-0.321470,-0.295670,0.44369,0.38995
78013,fraph,1.03880,-0.240660,0.556360,-0.240250,0.71568,-0.97994,0.642600,-0.650500,-0.089445,...,-0.153530,-0.585180,-0.516300,-0.782170,0.308690,-0.217080,-0.127580,0.133970,-0.25089,-0.71658
16695,mcdowell,-0.37532,-0.343330,-0.335120,0.984910,0.25654,0.25304,-0.546930,0.320400,-0.128310,...,-0.060907,0.688770,-0.508530,0.129620,-0.153290,0.295970,0.171770,-0.230180,-0.40340,0.45920
326348,nedelciu,-0.41663,-0.552270,-0.208120,0.155780,-0.50325,0.75483,-0.069079,0.058150,-0.106650,...,0.387740,-0.315890,-1.055000,0.063344,-0.167200,-0.462350,0.206650,0.274440,0.33568,-0.44969
116742,sli,0.27939,-0.706680,0.555550,1.102200,-0.46336,0.33452,1.124100,-0.632990,-0.440500,...,0.201820,-0.392730,-0.473270,0.139770,-0.389120,-0.851620,0.772090,0.200510,0.53654,0.56097


In [ ]:
import numpy as np
from typing import *

def _vectorize(word):
    word = word.lower()
    if word in df[0].tolist():
        vector = df[df[0] == word].iloc[:,1:]
        vector = vector.to_numpy().flatten()
    else:
        vector = np.zeros(df.shape[1]-1)
    return vector

def vectorize(words: Union[List[str], str]):
    if type(words) is list:
        words = list(map(_vectorize, words))
        words = np.stack(words, axis=0)
        return words
    return _vectorize(words)

def cossim(u, v):
    return u @ v / (np.linalg.norm(u) * np.linalg.norm(v))

In [ ]:
cossim(*vectorize('man game'.split())), cossim(*vectorize('woman game'.split())) # -> gender bias

(0.4837982694403796, 0.29868826318084823)

In [ ]:
g = vectorize('woman') - vectorize('man') # man -> woman

words = ['game', 'lipstick', 'guns', 'science', 'arts', 'literature',
         'warrior','doctor', 'tree', 'receptionist', 'technology', 
         'fashion', 'teacher', 'engineer', 'pilot', 'computer', 'singer']

for word in words:
    e = vectorize(word)
    sim = cossim(e,g)
    print(f"{word:15} {sim:>8.5f} {'♂' if sim < 0 else '♀'}")

game            -0.33507 ♂
lipstick         0.27692 ♀
guns            -0.18885 ♂
science         -0.06083 ♂
arts             0.00819 ♀
literature       0.06473 ♀
warrior         -0.20920 ♂
doctor           0.11895 ♀
tree            -0.07089 ♂
receptionist     0.33078 ♀
technology      -0.13194 ♂
fashion          0.03564 ♀
teacher          0.17921 ♀
engineer        -0.08039 ♂
pilot            0.00108 ♀
computer        -0.10330 ♂
singer           0.18501 ♀


### Debiasing technique

#### Neutralize
<img src="src/neutralize.png" width=800 />

In [ ]:
def neutralize(e, g): # debiasing non-gender-related words
    g_unit = g / np.linalg.norm(g)
    e_biased_factor = np.dot(e, g_unit) * g_unit
    e_debiased = e - e_biased_factor
    return e_debiased

g = vectorize('woman') - vectorize('man')
e = vectorize('game')

e_neutralized = neutralize(e, g)
cossim(e, g), cossim(e_neutralized, g) # near 0!

(-0.3350731148943847, 1.15603655367518e-16)

#### Equalize
<img src="src/equalize.png" width=800 />

In [ ]:
def equalize(e1, e2, g): # equalizing genderness of grammatical gender words
    e_mu = (e1 + e2) / 2
    g_unit = g / np.linalg.norm(g)
    e_mu_biased_factor = np.dot(e_mu, g_unit) * g_unit
    e_mu_orthogonal = e_mu - e_mu_biased_factor # lying on the g_orthogonal axis

    e1_biased_factor = np.dot(e1, g_unit) * g_unit # lying on the g axis
    e2_biased_factor = np.dot(e2, g_unit) * g_unit # lying on the g axis

    e1_corrected = e1_biased_factor + e_mu_orthogonal
    e2_corrected = e2_biased_factor + e_mu_orthogonal

    return e1_corrected, e2_corrected

g = vectorize('woman') - vectorize('man')
e1, e2 = vectorize('actor'), vectorize('actress')

e1_corrected, e2_corrected = equalize(e1, e2, g)

cossim(e2-e1, g), cossim(e2_corrected-e1_corrected, g) # near 1!

(0.7787298355385984, 0.9999999999999999)